<a href="https://colab.research.google.com/github/dinithprimal/TensorFlow_HandsOn/blob/main/Ex08_Tensorflow_2x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing

## RNN Poem/Play Generator

In [1]:
%tensorflow_version 2.x

from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### Dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### Loading Data

In [3]:
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

### Read Contents of File

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding

In [6]:
vocab = sorted(set(text))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
# Let's look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
# And here we will make a function that can convert our numeric values to text

def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Creating Training Examples

In [9]:
# Our first step will be to create a stream of characters from out text data

seq_length = 100 # length of sequence for a training example
example_per_epoch = len(text)//(seq_length + 1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
# Next we can use the batch method to turn this stream of characters into batches of desired lenght

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [11]:
# Now we need to use these sequences of length 101 and split them into input and output

def split_input_target(chunk):  # for the example: hello
  input_text = chunk[:-1]  # hell
  target_text = chunk[1:]  # ello
  return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [12]:
for x, y in dataset.take(2):
  print('\n\nEXAMPLE\n')
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
# Finally we need to make teaining batches

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite squence,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the Model

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(
          vocab_size,
          embedding_dim,
          batch_input_shape=[batch_size, None]
      ),
      tf.keras.layers.LSTM(
          rnn_units,
          return_sequences=True,
          stateful=True,
          recurrent_initializer='glorot_uniform'
      ),
      tf.keras.layers.Dense(vocab_size)

  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Creating a Loss Function

In [15]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for a prediction on out first batch of training data
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # print out the shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
# we can see that the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 2.1278937e-03  3.0047502e-03  4.5565674e-03 ...  1.4587980e-03
   -7.2790956e-04 -4.3709916e-03]
  [ 6.4861807e-03 -2.2537396e-03  3.6658992e-03 ...  3.7494134e-03
    1.0263856e-04 -2.8702314e-03]
  [ 8.8444427e-03 -2.2567031e-03  6.9050938e-03 ... -3.5881503e-03
   -6.1919284e-03  3.2912369e-03]
  ...
  [ 3.6886791e-03 -1.1928793e-02  4.4608433e-03 ...  1.4310230e-03
    1.0113213e-03  6.6531096e-03]
  [ 3.5818722e-03 -9.0815742e-03  1.3127537e-03 ... -4.6453113e-03
    3.3510458e-03  6.5625142e-03]
  [ 1.0883348e-03 -7.8963786e-03 -6.2328391e-03 ... -3.8814968e-03
    5.7682223e-03  5.3348620e-03]]

 [[-4.7675902e-03 -4.7176732e-03  1.5749282e-03 ...  3.2389574e-03
    3.6611832e-03 -6.0999068e-03]
  [ 4.5263121e-05 -6.5601366e-03  5.8718710e-03 ...  3.0717528e-03
    3.9296523e-03 -4.4152783e-03]
  [ 5.1376470e-03 -5.3713545e-03  7.6626013e-03 ... -2.4569300e-03
   -1.3814103e-03  2.3111580e-03]
  ...
  [-1.5195100e-02  2.8340169e-04 -1.1025434e-02 ...  8.7945694e

In [17]:
# let's examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2D array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00212789  0.00300475  0.00455657 ...  0.0014588  -0.00072791
  -0.00437099]
 [ 0.00648618 -0.00225374  0.0036659  ...  0.00374941  0.00010264
  -0.00287023]
 [ 0.00884444 -0.0022567   0.00690509 ... -0.00358815 -0.00619193
   0.00329124]
 ...
 [ 0.00368868 -0.01192879  0.00446084 ...  0.00143102  0.00101132
   0.00665311]
 [ 0.00358187 -0.00908157  0.00131275 ... -0.00464531  0.00335105
   0.00656251]
 [ 0.00108833 -0.00789638 -0.00623284 ... -0.0038815   0.00576822
   0.00533486]], shape=(100, 65), dtype=float32)


In [18]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probability of each character accuring next

65
tf.Tensor(
[ 0.00212789  0.00300475  0.00455657 -0.00336457 -0.00432321  0.00034721
  0.00118927 -0.00035017 -0.00702676 -0.00068105 -0.00453394 -0.00549321
 -0.00454005 -0.00347712 -0.00677659 -0.0017342   0.00234554  0.003143
  0.00266618  0.00218665  0.00045251 -0.00227905 -0.00145212 -0.00411965
 -0.00163225 -0.00215978  0.00505199  0.00265671 -0.00137684  0.00215418
 -0.00330329 -0.00345043 -0.0008038   0.00539538 -0.00147806 -0.00790671
 -0.00685127 -0.00170341 -0.00140664  0.00284512 -0.0014504  -0.00050722
  0.00086997 -0.00538255 -0.00225143  0.00241805  0.00496998 -0.00509916
  0.00518108 -0.00023781 -0.00053365  0.00109739  0.00341649  0.00294521
  0.00426318  0.00426968  0.00964413 -0.00349653  0.00483308 -0.00270794
  0.00266954  0.00148114  0.0014588  -0.00072791 -0.00437099], shape=(65,), dtype=float32)


In [19]:
# If we want to determine the predicted character we need to samle the output distribution (pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all tha integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"azkZ'p?,JpJ'BKcBrl-odh?p-YS;pFK3?Sx.IFqbNlwFofzQKamDjszFBXQc3ICapxD n!&hKN:\nqAxObHUCtpWCiKwez.x3TyRN"

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### Compiling the Model

In [21]:
model.compile(optimizer='adam', loss=loss)

### Creating Checkpoints

In [22]:
# Directinal where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

### Training

In [23]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - 20s 74ms/step - loss: 2.5744
Epoch 2/40
172/172 [==============================] - 13s 66ms/step - loss: 1.9003
Epoch 3/40
172/172 [==============================] - 13s 68ms/step - loss: 1.6498
Epoch 4/40
172/172 [==============================] - 14s 69ms/step - loss: 1.5123
Epoch 5/40
172/172 [==============================] - 14s 71ms/step - loss: 1.4290
Epoch 6/40
172/172 [==============================] - 14s 74ms/step - loss: 1.3720
Epoch 7/40
172/172 [==============================] - 15s 79ms/step - loss: 1.3267
Epoch 8/40
172/172 [==============================] - 17s 81ms/step - loss: 1.2868
Epoch 9/40
172/172 [==============================] - 15s 77ms/step - loss: 1.2508
Epoch 10/40
172/172 [==============================] - 14s 75ms/step - loss: 1.2162
Epoch 11/40
172/172 [==============================] - 15s 74ms/step - loss: 1.1814
Epoch 12/40
172/172 [==============================] - 15s 79ms/step - loss: 1.1465
E

### Loading the Model

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [25]:
# once the model is finished training we can find the latest checkpoint that stores the models weights using the following line

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [26]:
# We can load any checkpoint we want by specifying the exact file to load

# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint('./training_checkpoints/ckpt_' + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

### Generating text

In [27]:
def generated_text(model, start_string):
  # Evaluate step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our result
  text_generated = []

  # Low temperature results in more predictable text.
  # Higher temperature results in more surprising text
  # Experment to find the best setting
  temperature = 1.0

  # here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # Remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # Using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    # we pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generated_text(model, inp))

Type a starting string: Romeo 
Romeo pocket! his haste, and hast the time to prison.

GLOUCESTER:
Alas, that valiant you wot your faith in Paul,
Have no py sll pluck him home, that he hath too much of
you take our steed; know you what! by your father's voices?

MENENIUS:
Note me that made thyself, but saduted
With thy embracements, but suspecting an her sister'd from the seas.

KING RICHARD III:
Madam, how now! What! what! I am gower when I am, not
Richard, by Bolingbroke say I'll be you think if
He had some husband send to be of king.

GLOUCESTER:
Yes, tarry, are still combeth it for your pleasure; having no
ANGELOUSt the devil's dam:
And so, my gentle queen, parters, joy overture,
Which to the helps on the Tower and welcome.

MIRANDA:
How ble'd this knife be city,
To lick their heart to do it said 'Ay.'
And after tomp; one 
